## Step 1: Exogenous Shock
1. Create measure of code quality
   
    a. Measure of user-side code quality
   
    b. Measure of maintainer-side code quality
3. Analyze contributions made
4. Hypothesize how my instrument/exogenous affects either, and examine the empirical effect

## Step 2: 
How do I actually perform analysis?
1. Measure 1: Compare within repository groups
2. Measure 2: Find some other way to weight what a "download" means? 

In [187]:
import os
import pandas as pd
import ast
from pandarallel import pandarallel
import glob
import json
import numpy as np
from itertools import chain

In [2]:
pandarallel.initialize(progress_bar=False)

INFO: Pandarallel will run on 8 workers.
INFO: Pandarallel will use standard multiprocessing data transfer (pipe) to transfer data between the main process and workers.


In [3]:
pd.set_option('display.max_columns', None)

In [4]:
df_repo_info = pd.DataFrame()
df_actor_info = pd.DataFrame()
df_org_info = pd.DataFrame()

In [5]:
df_push = pd.DataFrame()
for i in range(99):
    if i < 10:
        i = f"0{i}"
    df_push_i = pd.read_csv(f'data/github_clean/pushEvent0000000000{i}.csv', index_col = 0)
    df_push = pd.concat([df_push_i[['type', 'created_at', 'repo_id', 'actor_id', 'org_id', 'push_id',
                                    'push_size', 'push_size_distinct', 'push_before', 'push_head']],
                         df_push])
    df_repo_i = df_push_i[['repo_id', 'repo_name']].drop_duplicates()
    df_actor_i = df_push_i[['actor_id', 'actor_login', 'repo_id', 'org_id',]].drop_duplicates()
    df_org_i = df_push_i[['org_id', 'org_login']].drop_duplicates()

    df_repo_info = pd.concat([df_repo_info, df_repo_i]).drop_duplicates()
    df_actor_info = pd.concat([df_actor_info, df_actor_i]).drop_duplicates()
    df_org_info = pd.concat([df_org_info, df_org_i]).drop_duplicates()

In [6]:
def cleanParquetPushes(df_parquet_repo):
    df_parquet_repo['ordering'] = df_parquet_repo.groupby('push_id').cumcount()+1
    df_parquet_repo = df_parquet_repo[['push_id', 'repo_id', 'actor_id','push_size', 'commit_groups',
                                       'commit sha', 'ordering', 'commit author name', 'commit author email',
                                       'committer name','commmitter email','commit message', 'commit additions',
                                       'commit deletions','commit changes total','commit files changed count',
                                       'commit file changes']]
    return df_parquet_repo

In [7]:
%%time
files = glob.glob("data/github_commits/parquet/*_push_*")
df_parquet_pushes_data = [cleanParquetPushes(pd.read_parquet(f,engine='fastparquet')) for f in files]
df_parquet_pushes = pd.concat(df_parquet_pushes_data,ignore_index=True)
df_parquet_pushes

CPU times: user 1min 10s, sys: 16.5 s, total: 1min 26s
Wall time: 1min 38s


,push_id,repo_id,actor_id,push_size,commit_groups,commit sha,ordering,commit author name,commit author email,committer name,commmitter email,commit message,commit additions,commit deletions,commit changes total,commit files changed count,commit file changes
0,8972291400,131384072,1658117,1,"['0a4393d9a8d856db1f2fec2d65ffeb86228b06d9', '...",0b2621d7f7f182300120384f598811a484e2dd12,1,Martin Thoma,info@martin-thoma.de,GitHub,noreply@github.com,MAINT: Update packages (#87)\n\n,14.0,14.0,28.0,6.0,"[{'file': '.pre-commit-config.yaml', 'addition..."
1,2936610255,131384072,1658117,1,"['f8a9d3076b8b421583b984dc6ba04686840f4d7e', '...",None,1,None,None,None,None,None,NaN,NaN,NaN,NaN,None
2,3927479739,131384072,1658117,1,"['802c5ac714c7cec1c48e5b68dce6d38c6c9c2471', '...",None,1,None,None,None,None,None,NaN,NaN,NaN,NaN,None
3,2847065656,131384072,1658117,1,"['c42aa1a8f48ca6d8be8360574ae36404c1a90545', '...",None,1,None,None,None,None,None,NaN,NaN,NaN,NaN,None
4,4153365390,131384072,1658117,1,"['e7212e8d250cd2372c1ceb3851042019bc724992', '...",None,1,None,None,None,None,None,NaN,NaN,NaN,NaN,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5082273,3210452736,62458409,12402673,2,"['2f04fd7014e7060ba1969db0b91392e396e0cfcb', '...",fad484976e3c4646fdff1782bc1fcf621fe76339,2,Raphael Meudec,raphael.meudec@gmail.com,GitHub,noreply@github.com,Merge pull request #286 from RaphaelMeudec/add...,0.0,0.0,0.0,0.0,[]
5082274,3025841603,62458409,15613912,2,"['e94ef887f5fb45d800d458f732e707a82c8b9b7d', '...",63cad1243c6924b7937accef3b3522ae765c7c81,1,Evan Hubinger,evanjhub@gmail.com,Evan Hubinger,evanjhub@gmail.com,Fix switching to new tf behavior\n,13.0,13.0,26.0,1.0,"[{'file': 'rl/agents/dqn.py', 'additions': 13,..."
5082275,3025841603,62458409,15613912,2,"['63cad1243c6924b7937accef3b3522ae765c7c81', '...",ac1439e7029b7d514aca53d3eecec5f64193d949,2,Rohan Raj,rajrohan1108@gmail.com,GitHub,noreply@github.com,Merge pull request #265 from evhub/master\n\nF...,0.0,0.0,0.0,0.0,[]
5082276,3821466181,62458409,15613912,2,"['e6efb0d8297ec38d704a3110b5d6ed74d09a05e3', '...",b74bf26c950eecc7fe6879c14937cb9b1ed22f90,1,Abhinav,abhinav.sagar2016@vitstudent.ac.in,Abhinav,abhinav.sagar2016@vitstudent.ac.in,Updated half cheetah environment\n,1.0,1.0,2.0,1.0,"[{'file': 'examples/ddpg_mujoco.py', 'addition..."


In [8]:
%%time
df_parquet_pushes['commit_groups'] = df_parquet_pushes['commit_groups'].parallel_apply(ast.literal_eval)
df_parquet_pushes['commit parent'] = df_parquet_pushes['commit_groups'].apply(lambda x: x[0] if len(x)>0 else '')

CPU times: user 9.98 s, sys: 3.78 s, total: 13.8 s
Wall time: 23.1 s


In [9]:
df_parquet_pushes.drop('commit_groups', axis = 1, inplace = True)

In [10]:
%%time
df_push['created_at'] = pd.to_datetime(df_push['created_at'])

CPU times: user 8.83 s, sys: 585 ms, total: 9.42 s
Wall time: 9.93 s


In [11]:
%%time
df_push_commits = pd.merge(df_push, df_parquet_pushes, how = 'left',
                           on = ['repo_id', 'push_id', 'actor_id'])

CPU times: user 10.8 s, sys: 13.4 s, total: 24.2 s
Wall time: 30.9 s


In [12]:
%%time
df_push_commits['push_day'] = df_push_commits['created_at'].parallel_apply(lambda x: x.day)
df_push_commits['push_month'] = df_push_commits['created_at'].parallel_apply(lambda x: x.month)
df_push_commits['push_year'] = df_push_commits['created_at'].parallel_apply(lambda x: x.year)

CPU times: user 535 ms, sys: 2.74 s, total: 3.28 s
Wall time: 9.36 s


In [13]:
df_push_commits = df_push_commits.rename({'push_size_x':'push_size'}, axis = 1).drop('push_size_y', axis = 1)

In [15]:
%%time
df_push_commits['commit file changes'] = df_push_commits['commit file changes'].apply(
    lambda x: [] if type(x) == float or type(x) == type(None) else x)

CPU times: user 1.61 s, sys: 38.3 ms, total: 1.65 s
Wall time: 1.66 s


In [221]:
df_push_commits_s = df_push_commits#.sample(100000)

In [ ]:
%%time
df_push_commits_s['commit file changes'] = df_push_commits_s['commit file changes'].parallel_apply(ast.literal_eval)

In [ ]:
def getList(x):
    try:
        return [ele['file'] for ele in x]
    except:
        return [ele['file'] for sublst in x for ele in sublst]

In [ ]:
%time
df_push_commits_grouped_daily = df_push_commits_s.groupby(['repo_id', 'push_year', 'push_month', 'push_day']).agg(
    unique_push_actors=('actor_id', lambda x: len(np.unique(x))),
    unique_push_orgs=('org_id', lambda x: len(np.unique(x))),
    push_size=('push_size', 'sum'),
    counted_commits=('commit sha', 'count'),
    retrieved_commits=('commit sha', lambda x: np.sum(type(x) == str)),
    unique_commit_authors=('commit author name', lambda x: len(np.unique(x.dropna()))),
    unique_commit_author_emails=('commit author email', lambda x: len(np.unique(x.dropna()))),
    unique_committers=('committer name', lambda x: len(np.unique(x.dropna()))),
    unique_committer_emails=('commmitter email', lambda x: len(np.unique(x.dropna()))),
    LOC_added=('commit additions', 'sum'),
    avg_LOC_added=('commit additions', 'mean'),
    LOC_deleted=('commit deletions', 'sum'),
    avg_LOC_changed=('commit deletions', 'mean'),
    files_changed=('commit files changed count', 'sum'),
    avg_files_changed=('commit files changed count', 'mean'),
    changed_files=('commit file changes', getList),
    uniq_changed_files=('commit file changes',  lambda x: len(getList(x)))
)

In [ ]:
df_push_commits_grouped_daily.to_parquet('data/aggregated_data/aggregated_daily_labor.parquet')

In [ ]:
%time
df_push_commits_grouped_monthly = df_push_commits_s.groupby(['repo_id', 'push_year', 'push_month']).agg(
    unique_push_actors=('actor_id', lambda x: len(np.unique(x))),
    unique_push_orgs=('org_id', lambda x: len(np.unique(x))),
    push_size=('push_size', 'sum'),
    counted_commits=('commit sha', 'count'),
    retrieved_commits=('commit sha', lambda x: np.sum(type(x) == str)),
    unique_commit_authors=('commit author name', lambda x: len(np.unique(x.dropna()))),
    unique_commit_author_emails=('commit author email', lambda x: len(np.unique(x.dropna()))),
    unique_committers=('committer name', lambda x: len(np.unique(x.dropna()))),
    unique_committer_emails=('commmitter email', lambda x: len(np.unique(x.dropna()))),
    LOC_added=('commit additions', 'sum'),
    avg_LOC_added=('commit additions', 'mean'),
    LOC_deleted=('commit deletions', 'sum'),
    avg_LOC_changed=('commit deletions', 'mean'),
    files_changed=('commit files changed count', 'sum'),
    avg_files_changed=('commit files changed count', 'mean'),
    changed_files=('commit file changes', getList),
    uniq_changed_files=('commit file changes',  lambda x: len(getList(x)))
)

In [ ]:
df_push_commits_grouped_monthly.to_parquet('data/aggregated_data/aggregated_daily_monthly.parquet')

In [215]:
"""
Two types of problems with pushes
1. push_size >= 250, so we have more pushes than we have commit data for
2. for some commits we have msising information 
"""
df_push_commits.head(5)

,type,created_at,repo_id,actor_id,org_id,push_id,push_size,push_size_distinct,push_before,push_head,commit sha,ordering,commit author name,commit author email,committer name,commmitter email,commit message,commit additions,commit deletions,commit changes total,commit files changed count,commit file changes,commit parent,push_day,push_month,push_year
0,PushEvent,2023-06-12 14:04:34+00:00,6235174,4546435,6392739.0,13953109285,1,1,07201cded25a27bd2514625c6e07030d9c273f2b,8f16c19ed7f1e69ab4e793b02833791f3f5b5535,8f16c19ed7f1e69ab4e793b02833791f3f5b5535,1.0,conda-bot,18747875+conda-bot@users.noreply.github.com,GitHub,noreply@github.com,Update test durations (#12783)\n\n,2404.0,1962.0,4366.0,3.0,"[{'file': 'tools/durations/Linux.json', 'addit...",07201cded25a27bd2514625c6e07030d9c273f2b,12,6,2023
1,PushEvent,2023-03-29 05:55:45+00:00,79536820,389236,NaN,13111219211,1,1,ff47d8f4c0794cde54e2b53274dc02792938c61a,8a6949cdddbe9c70fb4c434bda7aa62f1222a499,8a6949cdddbe9c70fb4c434bda7aa62f1222a499,1.0,Ashish Bhatia,ashishb@ashishb.net,Ashish Bhatia,ashishb@ashishb.net,Add repology badge\n,4.0,2.0,6.0,1.0,"[{'file': 'README.md', 'additions': 4, 'deleti...",ff47d8f4c0794cde54e2b53274dc02792938c61a,29,3,2023
2,PushEvent,2022-11-01 01:01:54+00:00,69691394,735008,NaN,11511181112,1,1,e58a899579ba416449c4e225b61f039457c8072a,d893bc56dd188d1961eda3ce9cbfb4544ea65613,d893bc56dd188d1961eda3ce9cbfb4544ea65613,1.0,pyup-bot,github-bot@pyup.io,pyup-bot,github-bot@pyup.io,Update pytz from 2021.1 to 2022.6,1.0,1.0,2.0,1.0,"[{'file': 'requirements/test.txt', 'additions'...",e58a899579ba416449c4e225b61f039457c8072a,1,11,2022
3,PushEvent,2023-03-31 19:20:50+00:00,91020334,8085768,5598284.0,13149043311,1,1,47ce8555cb7c210af7c79420e66d386959d9ca85,09ab9b1780d8d7eb35089aa9c21abe81537c582f,09ab9b1780d8d7eb35089aa9c21abe81537c582f,1.0,Jay Fitzgerald,34140133+ni-jfitzger@users.noreply.github.com,GitHub,noreply@github.com,Add set_runtime_environment to nidcpower Libra...,24.0,0.0,24.0,3.0,[{'file': 'generated/nidcpower/nidcpower/_grpc...,47ce8555cb7c210af7c79420e66d386959d9ca85,31,3,2023
4,PushEvent,2023-05-24 20:11:17+00:00,71932349,4529381,22125274.0,13750602561,1,1,f78626a49cbc3deb21b19cd6833486d5ef4f3c45,2980d0d3e0af53bb91948f29db0bdb0e741c8370,2980d0d3e0af53bb91948f29db0bdb0e741c8370,1.0,Cuong Nguyen,128072568+can-anyscale@users.noreply.github.com,GitHub,noreply@github.com,[ci][byod/6] build byod based on a commit hash...,131.0,49.0,180.0,3.0,"[{'file': 'release/ray_release/byod/build.py',...",f78626a49cbc3deb21b19cd6833486d5ef4f3c45,24,5,2023


In [220]:
df_push_commits.to_parquet('data/merged_data/push_commits.parquet')

In [ ]:
push, pull reqests data from data/github_commits/parquet

data/github_clean/pushEvent
data/github_clean/prReviewEvent
data/github_clean/prReviewCommentEvent
data/github_clean/prEventCommits
data/github_clean/prEvent
data/github_clean/issuesEvent
data/github_clean/issueCommentEvent